<a href="https://colab.research.google.com/github/anvo-2001/DAPortfolioProjects/blob/main/Weekly_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INTRODUCTION
* This Notebook is used to automatically query all KPIs for weekly, monthly, yearly report from Snowflake
* Input
  - Start, End Date: YYYY-MM-DD
  - Period: week, month, year
* Output: KPIs by period in a csv file

  **Use Crt+F9 to quick start the Code Execution after finishing Introduction**

# 1. Import packages, libraries and Set up connection to Database

In [2]:
# install pip snowflake package
!pip install snowflake-connector-python

# import library
import pandas as pd
import snowflake.connector as snowflake
import calendar
from datetime import date
import logging
# create connect to snowflake db
conn = snowflake.connect(
user        = "SAI_METABASE",
password    = "BWQ3n8wgMXgYr4s",
account     = "gameloft.eu-west-1",
db          = "ELEPHANT_DB",
importWarehouse = "GAME_TEAMS",
schema      = "DM2",
role = "GAME_TEAMS_WPO"
)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 4.4 MB/s eta 0:00:00


# 2. Execute SQL query and Retrieve Data

In [11]:
# Setup Logging function to report real-time code execution (debug mode) -> code will return value / result of each execution step
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create a handler that writes log messages to the notebook
handler = logging.StreamHandler()
handler.setLevel(logging.INFO)

# Create a formatter that adds contextual information to the log messages
formatter = logging.Formatter('Date-time: %(asctime)s - Line No.: %(lineno)%d - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(handler)

# Remove any default handlers to avoid duplicate logs
logger.handlers = [handler]

## All of the above code are for Jupyter's Logging function, if you are using Python app (VSCode or Pycharm) use:
    ### logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def query_execution(interval, start_date, end_date,connection):
  query = f"""
  -- DOWNLOADS
  with t1 as (
  SELECT date_trunc('{interval}', CLIENT_TIME) as period, sum(event_count) as Downloads
  from APPLAYDU_NOT_CERTIFIED.STORE_STATS t
  where 1=1
      and CLIENT_TIME >= '2020-08-10' and CLIENT_TIME < dateadd(day, -3, CURRENT_DATE())
      and CLIENT_TIME >= '{start_date}' and CLIENT_TIME < dateadd('{interval}',1,'{end_date}')
      and event_id = 393584
      and kpi_name in ('App Units','Install Events','Install events','New Downloads')
      and VERSION IN ('1.0.0')
  group by 1
  )
  -- Active Users
  , t2 as (
  select  date_trunc('{interval}',CLIENT_TIME) as period,count (distinct user_id) as "Active Users"
  FROM APPLAYDU.LAUNCH_RESUME
  WHERE 1=1
      and not (game_id = 82471 and CLIENT_TIME <'2020-12-14')
      and time_spent::float >= 0
      and time_spent::float < 86400
      and CLIENT_TIME >= '2020-08-10' and CLIENT_TIME < dateadd(day, -3, CURRENT_DATE())
      and CLIENT_TIME >= '{start_date}' and CLIENT_TIME < dateadd('{interval}',1,'{end_date}')
  group by 1
  )
  -- New Users
  , t3 as (
  SELECT date_trunc('{interval}',CLIENT_TIME) as period, count(distinct user_id) as "New Users"
  FROM APPLAYDU.LAUNCH_RESUME
  WHERE 1=1
      and session_id = 1
      and not(game_id = 82471 and client_time <'2020-12-14')
      and CLIENT_TIME >= '2020-08-10' and CLIENT_TIME < dateadd(day, -3, CURRENT_DATE())
      and CLIENT_TIME >= '{start_date}' and CLIENT_TIME < dateadd('{interval}',1,'{end_date}')
      and time_spent::float >= 0
      and time_spent::float < 86400
  group by 1
  )
  -- Sessions
  , t4 as (
  SELECT date_trunc('{interval}',CLIENT_TIME) as period, COUNT(*) as Sessions
  FROM APPLAYDU.LAUNCH_RESUME
  WHERE time_spent::float >= 0
      and time_spent::float < 86400
      and (session_id=1 or time_between_sessions::int>=30)
      and CLIENT_TIME >= '2020-08-10' and CLIENT_TIME < dateadd(day, -3, CURRENT_DATE())
      and CLIENT_TIME >= '{start_date}' and CLIENT_TIME < dateadd('{interval}',1,'{end_date}')
  group by 1
  )
  -- Sessions per User
  , t5 as (
  SELECT date_trunc('{interval}',CLIENT_TIME) as period
        , sum(case when (session_id=1 or time_between_sessions::int>=30) then 1 else 0 end) / COUNT(DISTINCT USER_ID) as "Average Session per User"
  FROM APPLAYDU.LAUNCH_RESUME
  WHERE time_spent::float >= 0
      and time_spent::float < 86400
      and CLIENT_TIME >= '2020-08-10' and CLIENT_TIME < dateadd(day, -3, CURRENT_DATE())
      and CLIENT_TIME >= '{start_date}' and CLIENT_TIME < dateadd('{interval}',1,'{end_date}')
  group by 1
  )

  -- Session Duration
  , t6 as (
  SELECT date_trunc('{interval}',CLIENT_TIME) as period
      , sum(time_spent::int) / sum(case when (session_id=1 or time_between_sessions::int>=30) then 1 else 0 end) as time_result
      , minute(time_result::int::string::time) || ' min '|| second(time_result::int::string::time) || ' sec ' as "Session Duration"
  FROM APPLAYDU.LAUNCH_RESUME
  WHERE time_spent::int >= 0
      and time_spent::int < 86400
      and CLIENT_TIME >= '2020-08-10' and CLIENT_TIME < dateadd(day, -3, CURRENT_DATE())
      and CLIENT_TIME >= '{start_date}' and CLIENT_TIME < dateadd('{interval}',1,'{end_date}')
  group by 1
  )
  -- Time spent by scan user
  , t7 as (
  select period
      ,sum(total_time_spent) / count(distinct user_id) as time_result
      ,hour(time_result::int::string::time) || ' hour '|| minute(time_result::int::string::time) || ' min '|| second(time_result::int::string::time) || ' sec ' as "Time spent per scan user"
  from (
  select user_id
      , date_trunc('{interval}',server_date) as period
      , sum (total_time_spent) as total_time_spent
      , sum (toy_unlocked_by_scan_count) as toy_unlocked_by_scan_count
      , sum (scan_mode_finished_count) as scan_mode_finished_count
  from APPLAYDU_NOT_CERTIFIED.tbl_users
  where 1=1
      and server_date < dateadd(day, -3, CURRENT_DATE())
      and server_date >= '{start_date}' and server_date < dateadd('{interval}',1,'{end_date}')
  group by 1,2
  ) as t_users
  where toy_unlocked_by_scan_count > 0 or scan_mode_finished_count > 0
  group by 1
  )
  -- Time spent by No Scan user
  , t8 as (
  select period
      , sum(total_time_spent) / count(distinct user_id) as time_result
      , hour(time_result::int::string::time) || ' hour '|| minute(time_result::int::string::time) || ' min '|| second(time_result::int::string::time) || ' sec ' as "Time spent per No scan user"
  from (
  select user_id
      , date_trunc('{interval}',server_date) as period
      , sum (total_time_spent) as total_time_spent
      , sum (toy_unlocked_by_scan_count) as toy_unlocked_by_scan_count
      , sum (scan_mode_finished_count) as scan_mode_finished_count
  from APPLAYDU_NOT_CERTIFIED.tbl_users
  where 1=1
      and server_date < dateadd(day, -3, CURRENT_DATE())
      and server_date >= '{start_date}' and server_date < dateadd('{interval}',1,'{end_date}')
  group by 1,2
  ) as t_users
  where toy_unlocked_by_scan_count = 0 or scan_mode_finished_count = 0
  group by 1
  )
  -- Number of Scans
  , t9 as (
  select date_trunc('{interval}',server_date) as period
    , sum (case when scan_type in ('Toy Scan','Scan_Toy') then total_scan else 0 end) as "Scan toy"
    , sum (case when scan_type in ('Scan_QR','QR Code') then total_scan else 0 end) as "QR Code"
    , sum (case when scan_type in ('Deep_Link') then total_scan else 0 end) as "Deep Link"
  from APPLAYDU_NOT_CERTIFIED.tbl_sum_scan_unlock t
  where scan_type in ('Alternative_Vignette','Scan_Vignette','Alternative Vignette','Vignette','Scan_QR','QR Code','Toy Scan','Scan_Toy','Deep_Link')
      and server_date < dateadd(day, -3, CURRENT_DATE())
      and server_date >= '{start_date}' and server_date < dateadd('{interval}',1,'{end_date}')
  group by 1
  )

  -- Scans per user
  , t10 as(
  select date_trunc('{interval}',server_date) as period
      , count (distinct user_id) as total_users
      , sum (toy_unlocked_by_scan_count) as sum_toy_unlocked_count
      , sum (scan_mode_finished_count) as sum_scan_mode_finished_count
      , (sum_toy_unlocked_count + sum_scan_mode_finished_count)/total_users as "Average Toys Scanned per User"
  from APPLAYDU_NOT_CERTIFIED.tbl_users
  where 1=1
      and server_date < dateadd(day, -3, CURRENT_DATE())
      and server_date >= '{start_date}' and server_date < dateadd('{interval}',1,'{end_date}')
      and (toy_unlocked_by_scan_count > 0 or scan_mode_finished_count > 0 )
  group by 1
  )
  SELECT
    t1.period,
    t1.Downloads,
    t2."Active Users",
    t3."New Users",
    t4.Sessions,
    t5."Average Session per User",
    t6."Session Duration",
    t7."Time spent per scan user",
    t8."Time spent per No scan user",
    t9."Scan toy",
    t9."QR Code",
    t9."Deep Link",
    (t9."Scan toy" + t9."QR Code" + t9."Deep Link") as "Total Scan",
    t10."Average Toys Scanned per User"
  FROM t1
      JOIN t2 using (period)
      JOIN t3 using (period)
      JOIN t4 using (period)
      JOIN t5 using (period)
      JOIN t6 using (period)
      JOIN t7 using (period)
      JOIN t8 using (period)
      JOIN t9 using (period)
      JOIN t10 using (period)
  ORDER BY t1.period
  """
  try: # USING TRY/ EXCEPT to get the code return in Error message if there are wrong input
      logging.info(f"Executing query for interval: {interval}, start_date: {start_date}, end_date: {end_date}")
      df = pd.read_sql_query(query, connection)

      if df.empty:
          logging.warning("Query returned an empty DataFrame.") #sometimes Snowflake will be lag and block access, if this message is displayed, run the code again after 15 mins
      else:
          logging.info("Query executed successfully and returned data.")

      return df

  except Exception as e:
      logging.error(f"An error occurred: {e}")
      return None
# Execute the function with Input value
interval = str(input('Interval of time period (ex: month, week, day, year):'))
if not interval or interval in ('0'): # set default value for "interval"
  interval = 'month'
start_date = str(input('Start date (format: YYYY-MM-DD):'))
if not start_date or start_date in ('0'):  # set default value for "start_date"
  start_date = '2023-01-01'
end_date = str(input('End date (format: YYYY-MM-DD):'))
if not end_date or end_date in ('today','0','current day','Today','TODAY','Current day','Current Day','CURRENT DAY'):  # set default value for "end_date"
  end_date = str(date.today())
try:
    df = query_execution(interval, start_date, end_date, conn)

    df.to_csv('report_data.csv')
    # Download the file automatically
    # file_path = '/content/report_data.csv'
    # files.download(file_path)

except Exception as e:
    print("Error Input, please run the code again and input values with the correct format") # if input wrong value or format, this  message will be displayed

Interval of time period (ex: month, week, day, year):month
Start date (format: YYYY-MM-DD):2024-01-01
End date (format: YYYY-MM-DD):2024-02-01


--- Logging error ---
Traceback (most recent call last):
  File "/usr/lib/python3.10/logging/__init__.py", line 1100, in emit
    msg = self.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 943, in format
    return fmt.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 681, in format
    s = self.formatMessage(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 650, in formatMessage
    return self._style.format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 440, in format
    return self._format(record)
  File "/usr/lib/python3.10/logging/__init__.py", line 436, in _format
    return self._fmt % values
ValueError: unsupported format character '%' (0x25) at index 44
Call stack:
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dis

# 3. Execute SQL query and Retrieve Data